In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sqlite3

from src.main import main

# Approach

Use a Pre-trained LLM:
* GPT-3, GPT-3.5, or a lighter version like GPT-2 (well-suited for text generation tasks)
* Fine-tuning or adapting for specific tasks later possible

Steps:
1. HTML Processing: Extract relevant information from the HTML file.
    * Use BeautifulSoup or lxml in Python to parse and extract information from the HTML file.
2. Image Processing: Extract relevant information from the image:
    * Use image-to-text models like Tesseracts or pytesseract (OCR libraries) to extract text from the image.
    * Use OpenCV or PIL (Pillow) in Python to process the image and extract relevant information.
3. Summarize the image and HTML information and the prompt from the playwright test code using T5 model.
4. Python Processing: Parse the given playwright test code for previous step as a precondition.
5. Combine the extracted information with the prompt:
    * Combine the extracted information from the HTML and the image with the prompt for the language model.
6. Pass the combined input to the language model for generating the UI test code.

--> Run locally via notebook or script using the main.py

# Load Data

In [4]:
# Connect to the database
conn = sqlite3.connect('../data/raw/playwright_script.db')
cursor = conn.cursor()

res = cursor.execute("SELECT * FROM tests")
items = res.fetchall()

print("There are {} data.".format(len(items)))

There are 100 data.


In [5]:
# Check the first item
items[0]

('1.1',
 '[1.1] Öffne die Arbeitsmappe "Übersicht Messstellen" im Ordner "Gewässergüte".',
 '[1.1] Expected result: Die Arbeitsmappe wird geöffnet, der Analysekontext ist nicht sichtbar.',
 '.\\html\\1_1.html',
 '.\\screenshot\\1_1.png',
 '.\\test_script\\1_1.spec.ts')

# Run UI Test Generation locally (smaller model GPT-2)

In [6]:
def get_previous_id(id):
    test, step = map(int, id.split('.'))
    if step > 1:
        previous_id = f"{test}.{step - 1}"
    else:
        print(f"Test {test} has no more previous step. No context available.")

    return previous_id

In [7]:
def fetch_relevant_items(db_file, current_id):
    # Connect to SQLite database
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Get the previous ID
    previous_id = get_previous_id(current_id)

    # Prepare the SQL query to retrieve the desired rows
    query = 'SELECT * FROM tests WHERE id IN (?, ?)'
    cursor.execute(query, (current_id, previous_id))

    # Fetch all matching rows
    items = cursor.fetchall()

    # Close the connection
    conn.close()

    return items

In [8]:
def map_items_to_args(items):
    # The file paths from the test x-1 are used as context
    html_path = items[0][3]
    image_path = items[0][4]
    precondition_path = items[0][5]
    # The last step of the text x is used as prompt
    steps = items[1][1].split(']')
    prompt = steps[-1].strip()

    return {"html_path": html_path,
            "image_path": image_path,
            "precondition_path": precondition_path,
            "prompt": prompt}

Now we can run the main function with the extracted information, we will get the generated UI test code for the test with id 1.4.

In [9]:
# Select test ID and database file
db_file = '../data/raw/playwright_script.db'
current_id = '1.4'

In [ ]:
items = fetch_relevant_items(db_file, current_id)
args = map_items_to_args(items)

In [29]:
%%time
main(**args)

2024-06-19 00:41:34 [src.main:24] [INFO] >>>> Loading context...
2024-06-19 00:41:34 [src.data.html_processor:27] [DEBUG] >>>> HTML content parsed successfully. - Lines of Code: 40
2024-06-19 00:41:35 [src.data.image_processor:26] [DEBUG] >>>> Text extracted from image successfully. - Characters: 103)
2024-06-19 00:41:35 [src.data.python_processor:15] [DEBUG] >>>> Python code parsed successfully. - Lines of Code: 17)
2024-06-19 00:41:35 [src.main:32] [INFO] >>>> Context loaded successfully.
2024-06-19 00:41:35 [src.main:34] [INFO] >>>> Generating test case...


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


2024-06-19 00:41:40 [src.ui_tests.test_generation:36] [DEBUG] >>>> Generated code saved to './pred_test_script/1_4.pred.py'
2024-06-19 00:41:40 [src.main:41] [INFO] >>>> Test case generated for 1_4.
CPU times: total: 2.36 s
Wall time: 6.41 s


# More Developing: Using modular source code
Possible to update function content in the following files, but no RENAMING or DELETING or changing function signature!:
* src/main.py: Main function for UI test generation.
    * Change constant Max length (max length of the input text for the model)
* src/ui_tests/test_generation.py: Generate the UI test code using the language model.
    * Expand model selection: Just add new case statements for different models with its code.
* src/data/html_processing.py: Extract relevant information from the HTML file.
* src/data/image_processing.py: Extract relevant information from the image.
* src/data/python_processing.py: Parse the given playwright test code for previous step as a precondition.
